<a href="https://colab.research.google.com/github/PatelHarshitt/DSP-PROJECTS/blob/main/Video_File_Heart_Rate_Monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq
import time

# --- Configuration ---
# !!! IMPORTANT: Place your video file in the same folder as this script
# and update the filename below.
VIDEO_FILENAME = "meow.mp4"  # <--- CHANGE THIS TO YOUR VIDEO FILE

# ROI selection: A square in the center of the frame.
# You may need to adjust this based on your video's resolution and where
# your finger is placed.
ROI_SIZE = 700

# Heart rate calculation parameters
BUFFER_SIZE = 250  # Number of frames to store for FFT analysis
MIN_HR_HZ = 40.0 / 60.0  # Minimum heart rate in Hz (40 BPM)
MAX_HR_HZ = 200.0 / 60.0 # Maximum heart rate in Hz (200 BPM)

# --- DSP Functions ---

def create_bandpass_filter(low, high, fs, order=5):
    """Creates a Butterworth bandpass filter."""
    nyq = 0.5 * fs
    low_norm = low / nyq
    high_norm = high / nyq
    b, a = butter(order, [low_norm, high_norm], btype='band')
    return b, a

def apply_filter(data, b, a):
    """Applies a filter to the data."""
    return filtfilt(b, a, data)

# --- Main Application Logic ---

def main():
    # Initialize video capture from the specified file
    cap = cv2.VideoCapture(VIDEO_FILENAME)
    if not cap.isOpened():
        print(f"Error: Cannot open video file '{VIDEO_FILENAME}'.")
        print("Please make sure the file is in the same folder as the script and the filename is correct.")
        return

    # Get video properties
    fs = cap.get(cv2.CAP_PROP_FPS)
    if fs == 0:
        print("Warning: Could not get FPS from video file. Assuming 30.0 FPS.")
        fs = 30.0 # Default fallback FPS (30/60)
    else:print(fs)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate ROI coordinates to be in the center
    roi_x = (frame_width - ROI_SIZE) // 2
    roi_y = (frame_height - ROI_SIZE) // 2

    # Data buffer to store the raw signal
    signal_buffer = []

    # Create the bandpass filter once
    b, a = create_bandpass_filter(MIN_HR_HZ, MAX_HR_HZ, fs, order=2)

    # Variables for BPM calculation and display
    bpm = 0.0
    bpm_values = [] # Store all calculated BPMs to average at the end

    print(f"Processing video file: {VIDEO_FILENAME}")
    print(f"Video properties: {frame_width}x{frame_height} @ {fs:.2f} FPS")
    print("Press 'q' to quit early.")

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            # End of video
            break

        frame_count += 1

        # --- Step 1: Define and draw the Region of Interest (ROI) ---
        roi = frame[roi_y:roi_y + ROI_SIZE, roi_x:roi_x + ROI_SIZE]
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + ROI_SIZE, roi_y + ROI_SIZE), (0, 255, 0), 2)

        # --- Step 2: Extract the Signal ---
        # We use the mean of the red channel.
        red_channel_mean = np.mean(roi[:, :, 2])
        signal_buffer.append(red_channel_mean)

        # Keep the buffer at a fixed size
        if len(signal_buffer) > BUFFER_SIZE:
            signal_buffer.pop(0)

        # --- Step 3: Process the signal when we have enough data ---
        if len(signal_buffer) == BUFFER_SIZE:
            # Detrend the signal
            detrended_signal = signal_buffer - np.mean(signal_buffer)

            # --- Step 4: Apply the Bandpass Filter ---
            filtered_signal = apply_filter(detrended_signal, b, a)

            # --- Step 5: Calculate Heart Rate using FFT ---
            N = len(filtered_signal)
            yf = fft(filtered_signal)
            xf = fftfreq(N, 1 / fs)

            # Find the dominant frequency in our valid heart rate range
            mask = (xf > MIN_HR_HZ) & (xf < MAX_HR_HZ)
            if np.any(mask):
                peak_freq = xf[mask][np.argmax(np.abs(yf[mask]))]
                bpm = peak_freq * 60.0
                bpm_values.append(bpm)

            # This creates a rolling window effect for smoother updates
            signal_buffer = signal_buffer[int(fs/4):]


        # --- Step 6: Display the output ---
        progress = frame_count / total_frames
        cv2.rectangle(frame, (0, frame_height - 10), (int(frame_width * progress), frame_height), (0, 255, 0), -1)

        status_text = f"Processing frame {frame_count}/{total_frames}"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        bpm_text = f"Current BPM: {bpm:.1f}"
        cv2.putText(frame, bpm_text, (roi_x, roi_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the final frame
        cv2_imshow(frame)

        # Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # --- Final Calculation and Cleanup ---
    if bpm_values:
        average_bpm = np.mean(bpm_values)
        print("\n--- Processing Complete ---")
        print(f"Average Heart Rate: {average_bpm:.2f} BPM")
    else:
        print("\n--- Processing Complete ---")
        print("Could not calculate a heart rate. The video might be too short or the signal was not clear.")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()